<a href="https://colab.research.google.com/github/narcisonascimento/pyspark-estudos/blob/main/pyspark_estudos_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pyspark Estudos <br>
Fundamentos do Pyspark <br>
Professor: Renan Marques Rodrigues <br>
Youtube: DataDev Academy | https://www.youtube.com/@datadevacademy <br>
Dataset: https://www.kaggle.com/datasets/djamshed/fifa-world-cup-2018-players

# Aula 1

# Instalando bibliotecas

In [1]:
# instalando pyspark
# !pip install pyspark

In [2]:
# pip freeze

# Importando Bibliotecas

In [3]:
import os
import sys

from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

# os.environ['PYSPARK_PYTHON'] = sys.executable
# os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

# Criar / iniciar Sessão PySpark

In [4]:
spark = (
    SparkSession.builder
    .master('local')
    .appName('pyspark-estudos')
    .getOrCreate()
)

# Criar DF / ler arquivo

# Verificar tipos de colunas

In [5]:
df = spark.read.csv('/content/wc2018-players.csv', header=True, inferSchema=True)
df.show(5)

+---------+---+----+------------------+----------+----------+--------------------+------+------+
|     Team|  #|Pos.| FIFA Popular Name|Birth Date|Shirt Name|                Club|Height|Weight|
+---------+---+----+------------------+----------+----------+--------------------+------+------+
|Argentina|  3|  DF|TAGLIAFICO Nicolas|31.08.1992|TAGLIAFICO|      AFC Ajax (NED)|   169|    65|
|Argentina| 22|  MF|    PAVON Cristian|21.01.1996|     PAVÓN|CA Boca Juniors (...|   169|    65|
|Argentina| 15|  MF|    LANZINI Manuel|15.02.1993|   LANZINI|West Ham United F...|   167|    66|
|Argentina| 18|  DF|    SALVIO Eduardo|13.07.1990|    SALVIO|    SL Benfica (POR)|   167|    69|
|Argentina| 10|  FW|      MESSI Lionel|24.06.1987|     MESSI|  FC Barcelona (ESP)|   170|    72|
+---------+---+----+------------------+----------+----------+--------------------+------+------+
only showing top 5 rows



In [6]:
df.printSchema()

root
 |-- Team: string (nullable = true)
 |-- #: integer (nullable = true)
 |-- Pos.: string (nullable = true)
 |-- FIFA Popular Name: string (nullable = true)
 |-- Birth Date: string (nullable = true)
 |-- Shirt Name: string (nullable = true)
 |-- Club: string (nullable = true)
 |-- Height: integer (nullable = true)
 |-- Weight: integer (nullable = true)



# Verificar dados nulos

In [7]:
# transformando em Pandas e efetuando a soma. Aqui só funciona com poucas linhas no dataset
df.toPandas().isna().sum()

Team                 0
#                    0
Pos.                 0
FIFA Popular Name    0
Birth Date           0
Shirt Name           0
Club                 0
Height               0
Weight               0
dtype: int64

In [8]:
# verificando as colunas existentes
df.columns

['Team',
 '#',
 'Pos.',
 'FIFA Popular Name',
 'Birth Date',
 'Shirt Name',
 'Club',
 'Height',
 'Weight']

In [9]:
# No PySpark deve ser feito um looping for, esse looping demora dependendo do tamanho do dataset
# Este for apresentará erro por conta do nome da coluna onde Pos. tem um ponto.
for coluna in df.columns:
    print(coluna, df.filter(df[coluna].isNull()).count())

Team 0
# 0


AnalysisException: Syntax error in attribute name: Pos..

# Renomear colunas <br>
PySpark não aceita pontos no nome e também deve ter atenção com espaço entre palavras.

In [10]:
df = df.withColumnRenamed('Team', 'selecao')\
.withColumnRenamed('#', 'numero')\
.withColumnRenamed('Pos.', 'posicao')\
.withColumnRenamed('FIFA Popular Name', 'nome_fifa')\
.withColumnRenamed('Birth Date', 'data_nasc')\
.withColumnRenamed('Shirt Name', 'nome_camisa')\
.withColumnRenamed('Club', 'time')\
.withColumnRenamed('Height', 'altura')\
.withColumnRenamed('Weight', 'peso')

In [11]:
df.columns

['selecao',
 'numero',
 'posicao',
 'nome_fifa',
 'data_nasc',
 'nome_camisa',
 'time',
 'altura',
 'peso']

In [12]:
# após as devidas correções é possível efetuar o looping no dataset

for coluna in df.columns:
    print(coluna, df.filter(df[coluna].isNull()).count())

selecao 0
numero 0
posicao 0
nome_fifa 0
data_nasc 0
nome_camisa 0
time 0
altura 0
peso 0


# Selecionar colunas

In [13]:
# diferente do Pandas o PySpark altera o nome da coluna original quando escrita de forma diferente (caps lock etc).
df.select('selecao', 'nome_fifa').show(5)

+---------+------------------+
|  selecao|         nome_fifa|
+---------+------------------+
|Argentina|TAGLIAFICO Nicolas|
|Argentina|    PAVON Cristian|
|Argentina|    LANZINI Manuel|
|Argentina|    SALVIO Eduardo|
|Argentina|      MESSI Lionel|
+---------+------------------+
only showing top 5 rows



In [14]:
df.select('SELECAO', 'NOME_FIFA').show(5)

+---------+------------------+
|  SELECAO|         NOME_FIFA|
+---------+------------------+
|Argentina|TAGLIAFICO Nicolas|
|Argentina|    PAVON Cristian|
|Argentina|    LANZINI Manuel|
|Argentina|    SALVIO Eduardo|
|Argentina|      MESSI Lionel|
+---------+------------------+
only showing top 5 rows



Pode selecionar colunas com o 'col'

In [15]:
df.select(col('selecao'), col('nome_camisa'), col('altura')).show(5)

+---------+-----------+------+
|  selecao|nome_camisa|altura|
+---------+-----------+------+
|Argentina| TAGLIAFICO|   169|
|Argentina|      PAVÓN|   169|
|Argentina|    LANZINI|   167|
|Argentina|     SALVIO|   167|
|Argentina|      MESSI|   170|
+---------+-----------+------+
only showing top 5 rows



ou pode selecioar com o dataframe['nome_coluna']

In [16]:
df.select(df['selecao'], df['nome_camisa'], df['altura']).show(5)

+---------+-----------+------+
|  selecao|nome_camisa|altura|
+---------+-----------+------+
|Argentina| TAGLIAFICO|   169|
|Argentina|      PAVÓN|   169|
|Argentina|    LANZINI|   167|
|Argentina|     SALVIO|   167|
|Argentina|      MESSI|   170|
+---------+-----------+------+
only showing top 5 rows



# Selecionar Colunas com ALIAS

In [17]:
df.select(col('selecao').alias('time')).show(5)

+---------+
|     time|
+---------+
|Argentina|
|Argentina|
|Argentina|
|Argentina|
|Argentina|
+---------+
only showing top 5 rows



In [18]:
# utilizando o split para separar as colunas
df.select('selecao nome_fifa altura'.split()).show(5)

+---------+------------------+------+
|  selecao|         nome_fifa|altura|
+---------+------------------+------+
|Argentina|TAGLIAFICO Nicolas|   169|
|Argentina|    PAVON Cristian|   169|
|Argentina|    LANZINI Manuel|   167|
|Argentina|    SALVIO Eduardo|   167|
|Argentina|      MESSI Lionel|   170|
+---------+------------------+------+
only showing top 5 rows



# Organizar o Select

In [19]:
df.select('nome_camisa', 'altura', 'peso').show(5)

+-----------+------+----+
|nome_camisa|altura|peso|
+-----------+------+----+
| TAGLIAFICO|   169|  65|
|      PAVÓN|   169|  65|
|    LANZINI|   167|  66|
|     SALVIO|   167|  69|
|      MESSI|   170|  72|
+-----------+------+----+
only showing top 5 rows



# Filtrar o dataframe

In [20]:
df.filter('selecao = "Brazil"').show(10)

+-------+------+-------+-----------------+----------+-----------+--------------------+------+----+
|selecao|numero|posicao|        nome_fifa| data_nasc|nome_camisa|                time|altura|peso|
+-------+------+-------+-----------------+----------+-----------+--------------------+------+----+
| Brazil|    18|     MF|             FRED|05.03.1993|       FRED|FC Shakhtar Donet...|   169|  64|
| Brazil|    21|     FW|           TAISON|13.01.1988|     TAISON|FC Shakhtar Donet...|   172|  64|
| Brazil|    17|     MF|      FERNANDINHO|04.05.1985|FERNANDINHO|Manchester City F...|   179|  67|
| Brazil|    22|     DF|           FAGNER|11.06.1989|     FAGNER|SC Corinthians (BRA)|   168|  67|
| Brazil|    10|     FW|           NEYMAR|05.02.1992|  NEYMAR JR|Paris Saint-Germa...|   175|  68|
| Brazil|    11|     MF|PHILIPPE COUTINHO|12.06.1992|P. COUTINHO|  FC Barcelona (ESP)|   172|  68|
| Brazil|     7|     FW|    DOUGLAS COSTA|14.09.1990|   D. COSTA|   Juventus FC (ITA)|   182|  70|
| Brazil| 

In [21]:
# caso o nome da coluna possua espaço, não pode fazer o filter direto como abaixo, deve chamar a função col
df.filter('nome_camisa == "FRED"').show()

+-------+------+-------+---------+----------+-----------+--------------------+------+----+
|selecao|numero|posicao|nome_fifa| data_nasc|nome_camisa|                time|altura|peso|
+-------+------+-------+---------+----------+-----------+--------------------+------+----+
| Brazil|    18|     MF|     FRED|05.03.1993|       FRED|FC Shakhtar Donet...|   169|  64|
+-------+------+-------+---------+----------+-----------+--------------------+------+----+



In [22]:
df.filter(col('nome_camisa') == 'FRED').show()

+-------+------+-------+---------+----------+-----------+--------------------+------+----+
|selecao|numero|posicao|nome_fifa| data_nasc|nome_camisa|                time|altura|peso|
+-------+------+-------+---------+----------+-----------+--------------------+------+----+
| Brazil|    18|     MF|     FRED|05.03.1993|       FRED|FC Shakhtar Donet...|   169|  64|
+-------+------+-------+---------+----------+-----------+--------------------+------+----+



# Filtrar DF com 2 condições (and/&)

In [23]:
df.filter((col('selecao') == "England") & (col('altura') >= 180) & (col('peso') >= 82)).show()

+-------+------+-------+----------------+----------+-----------+--------------------+------+----+
|selecao|numero|posicao|       nome_fifa| data_nasc|nome_camisa|                time|altura|peso|
+-------+------+-------+----------------+----------+-----------+--------------------+------+----+
|England|     8|     MF|HENDERSON Jordan|17.06.1990|  HENDERSON|  Liverpool FC (ENG)|   183|  82|
|England|    15|     DF|     CAHILL Gary|19.12.1985|     CAHILL|    Chelsea FC (ENG)|   191|  85|
|England|     4|     MF|       DIER Eric|15.01.1994|       DIER|Tottenham Hotspur...|   188|  90|
|England|    13|     GK|    BUTLAND Jack|10.03.1993|    BUTLAND| Stoke City FC (ENG)|   196|  96|
|England|     6|     DF|   MAGUIRE Harry|05.03.1993|    MAGUIRE|Leicester City FC...|   193|  98|
|England|     9|     FW|      KANE Harry|28.07.1993|       KANE|Tottenham Hotspur...|   188|  98|
+-------+------+-------+----------------+----------+-----------+--------------------+------+----+



In [24]:
df.filter('selecao = "Brazil"').filter(col('numero') > 15).show()

+-------+------+-------+---------------+----------+-----------+--------------------+------+----+
|selecao|numero|posicao|      nome_fifa| data_nasc|nome_camisa|                time|altura|peso|
+-------+------+-------+---------------+----------+-----------+--------------------+------+----+
| Brazil|    18|     MF|           FRED|05.03.1993|       FRED|FC Shakhtar Donet...|   169|  64|
| Brazil|    21|     FW|         TAISON|13.01.1988|     TAISON|FC Shakhtar Donet...|   172|  64|
| Brazil|    17|     MF|    FERNANDINHO|04.05.1985|FERNANDINHO|Manchester City F...|   179|  67|
| Brazil|    22|     DF|         FAGNER|11.06.1989|     FAGNER|SC Corinthians (BRA)|   168|  67|
| Brazil|    20|     FW|ROBERTO FIRMINO|02.10.1991|    FIRMINO|  Liverpool FC (ENG)|   181|  76|
| Brazil|    19|     MF|        WILLIAN|09.08.1988|    WILLIAN|    Chelsea FC (ENG)|   175|  77|
| Brazil|    23|     GK|        EDERSON|17.08.1993|    EDERSON|Manchester City F...|   188|  86|
| Brazil|    16|     GK|      

# Filter df com 2 condições (or/|)

In [25]:
df.filter((col('nome_fifa') == 'TAISON') | (col('nome_fifa') == 'MESSI Lionel')).show()

+---------+------+-------+------------+----------+-----------+--------------------+------+----+
|  selecao|numero|posicao|   nome_fifa| data_nasc|nome_camisa|                time|altura|peso|
+---------+------+-------+------------+----------+-----------+--------------------+------+----+
|Argentina|    10|     FW|MESSI Lionel|24.06.1987|      MESSI|  FC Barcelona (ESP)|   170|  72|
|   Brazil|    21|     FW|      TAISON|13.01.1988|     TAISON|FC Shakhtar Donet...|   172|  64|
+---------+------+-------+------------+----------+-----------+--------------------+------+----+



# Filter df combinando & e | (and e Or)

In [26]:
df.filter((col('selecao') == 'Brazil') & (col('posicao') == 'DF') | (col('nome_fifa') == 'MESSI Lionel')).show()

+---------+------+-------+-------------+----------+-----------+--------------------+------+----+
|  selecao|numero|posicao|    nome_fifa| data_nasc|nome_camisa|                time|altura|peso|
+---------+------+-------+-------------+----------+-----------+--------------------+------+----+
|Argentina|    10|     FW| MESSI Lionel|24.06.1987|      MESSI|  FC Barcelona (ESP)|   170|  72|
|   Brazil|    22|     DF|       FAGNER|11.06.1989|     FAGNER|SC Corinthians (BRA)|   168|  67|
|   Brazil|     6|     DF|  FILIPE LUIS|09.08.1985|FILIPE LUIS|Atletico Madrid (...|   182|  73|
|   Brazil|    13|     DF|   MARQUINHOS|14.05.1994| MARQUINHOS|Paris Saint-Germa...|   183|  75|
|   Brazil|     3|     DF|      MIRANDA|07.09.1984|    MIRANDA|FC Internazionale...|   186|  78|
|   Brazil|    14|     DF|       DANILO|15.07.1991|     DANILO|Manchester City F...|   184|  78|
|   Brazil|     2|     DF| THIAGO SILVA|22.09.1984|   T. SILVA|Paris Saint-Germa...|   183|  79|
|   Brazil|    12|     DF|    

In [27]:
df.filter((col('selecao') == 'Brazil') & (col('posicao') == 'DF') | (col('altura') == 199) & (col('selecao') == 'Belgium')).show()

+-------+------+-------+----------------+----------+-----------+--------------------+------+----+
|selecao|numero|posicao|       nome_fifa| data_nasc|nome_camisa|                time|altura|peso|
+-------+------+-------+----------------+----------+-----------+--------------------+------+----+
|Belgium|     1|     GK|COURTOIS Thibaut|11.05.1992|   COURTOIS|    Chelsea FC (ENG)|   199|  91|
| Brazil|    22|     DF|          FAGNER|11.06.1989|     FAGNER|SC Corinthians (BRA)|   168|  67|
| Brazil|     6|     DF|     FILIPE LUIS|09.08.1985|FILIPE LUIS|Atletico Madrid (...|   182|  73|
| Brazil|    13|     DF|      MARQUINHOS|14.05.1994| MARQUINHOS|Paris Saint-Germa...|   183|  75|
| Brazil|     3|     DF|         MIRANDA|07.09.1984|    MIRANDA|FC Internazionale...|   186|  78|
| Brazil|    14|     DF|          DANILO|15.07.1991|     DANILO|Manchester City F...|   184|  78|
| Brazil|     2|     DF|    THIAGO SILVA|22.09.1984|   T. SILVA|Paris Saint-Germa...|   183|  79|
| Brazil|    12|    

# Criar novas colunas usando a função lit

In [28]:
df.withColumn('worldcup', lit(2018)).show(5)

+---------+------+-------+------------------+----------+-----------+--------------------+------+----+--------+
|  selecao|numero|posicao|         nome_fifa| data_nasc|nome_camisa|                time|altura|peso|worldcup|
+---------+------+-------+------------------+----------+-----------+--------------------+------+----+--------+
|Argentina|     3|     DF|TAGLIAFICO Nicolas|31.08.1992| TAGLIAFICO|      AFC Ajax (NED)|   169|  65|    2018|
|Argentina|    22|     MF|    PAVON Cristian|21.01.1996|      PAVÓN|CA Boca Juniors (...|   169|  65|    2018|
|Argentina|    15|     MF|    LANZINI Manuel|15.02.1993|    LANZINI|West Ham United F...|   167|  66|    2018|
|Argentina|    18|     DF|    SALVIO Eduardo|13.07.1990|     SALVIO|    SL Benfica (POR)|   167|  69|    2018|
|Argentina|    10|     FW|      MESSI Lionel|24.06.1987|      MESSI|  FC Barcelona (ESP)|   170|  72|    2018|
+---------+------+-------+------------------+----------+-----------+--------------------+------+----+--------+
o

In [29]:
df.withColumn('coluna_nova', lit(col('altura') - col('peso'))).show(5)

+---------+------+-------+------------------+----------+-----------+--------------------+------+----+-----------+
|  selecao|numero|posicao|         nome_fifa| data_nasc|nome_camisa|                time|altura|peso|coluna_nova|
+---------+------+-------+------------------+----------+-----------+--------------------+------+----+-----------+
|Argentina|     3|     DF|TAGLIAFICO Nicolas|31.08.1992| TAGLIAFICO|      AFC Ajax (NED)|   169|  65|        104|
|Argentina|    22|     MF|    PAVON Cristian|21.01.1996|      PAVÓN|CA Boca Juniors (...|   169|  65|        104|
|Argentina|    15|     MF|    LANZINI Manuel|15.02.1993|    LANZINI|West Ham United F...|   167|  66|        101|
|Argentina|    18|     DF|    SALVIO Eduardo|13.07.1990|     SALVIO|    SL Benfica (POR)|   167|  69|         98|
|Argentina|    10|     FW|      MESSI Lionel|24.06.1987|      MESSI|  FC Barcelona (ESP)|   170|  72|         98|
+---------+------+-------+------------------+----------+-----------+--------------------

# Criar coluna condicional usando função substring

In [30]:
df.withColumn('sub', substring('selecao', 1, 3)).show(5)

+---------+------+-------+------------------+----------+-----------+--------------------+------+----+---+
|  selecao|numero|posicao|         nome_fifa| data_nasc|nome_camisa|                time|altura|peso|sub|
+---------+------+-------+------------------+----------+-----------+--------------------+------+----+---+
|Argentina|     3|     DF|TAGLIAFICO Nicolas|31.08.1992| TAGLIAFICO|      AFC Ajax (NED)|   169|  65|Arg|
|Argentina|    22|     MF|    PAVON Cristian|21.01.1996|      PAVÓN|CA Boca Juniors (...|   169|  65|Arg|
|Argentina|    15|     MF|    LANZINI Manuel|15.02.1993|    LANZINI|West Ham United F...|   167|  66|Arg|
|Argentina|    18|     DF|    SALVIO Eduardo|13.07.1990|     SALVIO|    SL Benfica (POR)|   167|  69|Arg|
|Argentina|    10|     FW|      MESSI Lionel|24.06.1987|      MESSI|  FC Barcelona (ESP)|   170|  72|Arg|
+---------+------+-------+------------------+----------+-----------+--------------------+------+----+---+
only showing top 5 rows



In [31]:
df = df.withColumn('ano_nasc', substring('data_nasc', -4, 4))

# Criar coluna condicional usando concat / concat_ws

In [32]:
df.withColumn('concat', concat('selecao', col('nome_fifa'))).show(5)

+---------+------+-------+------------------+----------+-----------+--------------------+------+----+--------+--------------------+
|  selecao|numero|posicao|         nome_fifa| data_nasc|nome_camisa|                time|altura|peso|ano_nasc|              concat|
+---------+------+-------+------------------+----------+-----------+--------------------+------+----+--------+--------------------+
|Argentina|     3|     DF|TAGLIAFICO Nicolas|31.08.1992| TAGLIAFICO|      AFC Ajax (NED)|   169|  65|    1992|ArgentinaTAGLIAFI...|
|Argentina|    22|     MF|    PAVON Cristian|21.01.1996|      PAVÓN|CA Boca Juniors (...|   169|  65|    1996|ArgentinaPAVON Cr...|
|Argentina|    15|     MF|    LANZINI Manuel|15.02.1993|    LANZINI|West Ham United F...|   167|  66|    1993|ArgentinaLANZINI ...|
|Argentina|    18|     DF|    SALVIO Eduardo|13.07.1990|     SALVIO|    SL Benfica (POR)|   167|  69|    1990|ArgentinaSALVIO E...|
|Argentina|    10|     FW|      MESSI Lionel|24.06.1987|      MESSI|  FC Bar

In [33]:
df.withColumn('separador', concat_ws(' - ', 'selecao', 'numero', 'posicao')).show()

+---------+------+-------+------------------+----------+-----------+--------------------+------+----+--------+-------------------+
|  selecao|numero|posicao|         nome_fifa| data_nasc|nome_camisa|                time|altura|peso|ano_nasc|          separador|
+---------+------+-------+------------------+----------+-----------+--------------------+------+----+--------+-------------------+
|Argentina|     3|     DF|TAGLIAFICO Nicolas|31.08.1992| TAGLIAFICO|      AFC Ajax (NED)|   169|  65|    1992| Argentina - 3 - DF|
|Argentina|    22|     MF|    PAVON Cristian|21.01.1996|      PAVÓN|CA Boca Juniors (...|   169|  65|    1996|Argentina - 22 - MF|
|Argentina|    15|     MF|    LANZINI Manuel|15.02.1993|    LANZINI|West Ham United F...|   167|  66|    1993|Argentina - 15 - MF|
|Argentina|    18|     DF|    SALVIO Eduardo|13.07.1990|     SALVIO|    SL Benfica (POR)|   167|  69|    1990|Argentina - 18 - DF|
|Argentina|    10|     FW|      MESSI Lionel|24.06.1987|      MESSI|  FC Barcelona 

# Alterar o tipo de coluna

In [34]:
df.printSchema()

root
 |-- selecao: string (nullable = true)
 |-- numero: integer (nullable = true)
 |-- posicao: string (nullable = true)
 |-- nome_fifa: string (nullable = true)
 |-- data_nasc: string (nullable = true)
 |-- nome_camisa: string (nullable = true)
 |-- time: string (nullable = true)
 |-- altura: integer (nullable = true)
 |-- peso: integer (nullable = true)
 |-- ano_nasc: string (nullable = true)



In [35]:
df = df.withColumn('ano_nasc', col('ano_nasc').cast(IntegerType()))

In [36]:
df.printSchema()

root
 |-- selecao: string (nullable = true)
 |-- numero: integer (nullable = true)
 |-- posicao: string (nullable = true)
 |-- nome_fifa: string (nullable = true)
 |-- data_nasc: string (nullable = true)
 |-- nome_camisa: string (nullable = true)
 |-- time: string (nullable = true)
 |-- altura: integer (nullable = true)
 |-- peso: integer (nullable = true)
 |-- ano_nasc: integer (nullable = true)



# Desafio: Coluna Nascimento


*   Colocar como DataType
*   Dica: o formato percisa estar: YYYY-MM-DD



In [37]:
df.show(5)

+---------+------+-------+------------------+----------+-----------+--------------------+------+----+--------+
|  selecao|numero|posicao|         nome_fifa| data_nasc|nome_camisa|                time|altura|peso|ano_nasc|
+---------+------+-------+------------------+----------+-----------+--------------------+------+----+--------+
|Argentina|     3|     DF|TAGLIAFICO Nicolas|31.08.1992| TAGLIAFICO|      AFC Ajax (NED)|   169|  65|    1992|
|Argentina|    22|     MF|    PAVON Cristian|21.01.1996|      PAVÓN|CA Boca Juniors (...|   169|  65|    1996|
|Argentina|    15|     MF|    LANZINI Manuel|15.02.1993|    LANZINI|West Ham United F...|   167|  66|    1993|
|Argentina|    18|     DF|    SALVIO Eduardo|13.07.1990|     SALVIO|    SL Benfica (POR)|   167|  69|    1990|
|Argentina|    10|     FW|      MESSI Lionel|24.06.1987|      MESSI|  FC Barcelona (ESP)|   170|  72|    1987|
+---------+------+-------+------------------+----------+-----------+--------------------+------+----+--------+
o

In [42]:
# lamba para efetuar a separação pelo ponto, assim é possível pegar o valor pelo índice
# dia = udf(lambda x: x.split('.')[0])

dia = udf(lambda data: data.split('.')[0])
# incluindo coluna de dia no dataframe
df = df.withColumn('dia_nasc', dia('data_nasc'))

In [43]:
# incluindo coluna de mes no dataframe
mes = udf(lambda data: data.split('.')[1])
df = df.withColumn('mes_nasc', mes('data_nasc'))

In [45]:
df.show()

+---------+------+-------+------------------+----------+-----------+--------------------+------+----+--------+--------+--------+
|  selecao|numero|posicao|         nome_fifa| data_nasc|nome_camisa|                time|altura|peso|ano_nasc|dia_nasc|mes_nasc|
+---------+------+-------+------------------+----------+-----------+--------------------+------+----+--------+--------+--------+
|Argentina|     3|     DF|TAGLIAFICO Nicolas|31.08.1992| TAGLIAFICO|      AFC Ajax (NED)|   169|  65|    1992|      31|      08|
|Argentina|    22|     MF|    PAVON Cristian|21.01.1996|      PAVÓN|CA Boca Juniors (...|   169|  65|    1996|      21|      01|
|Argentina|    15|     MF|    LANZINI Manuel|15.02.1993|    LANZINI|West Ham United F...|   167|  66|    1993|      15|      02|
|Argentina|    18|     DF|    SALVIO Eduardo|13.07.1990|     SALVIO|    SL Benfica (POR)|   167|  69|    1990|      13|      07|
|Argentina|    10|     FW|      MESSI Lionel|24.06.1987|      MESSI|  FC Barcelona (ESP)|   170| 

In [47]:
df = df.withColumn('nascimento', concat_ws('-', 'ano_nasc', "mes_nasc", "dia_nasc"))

In [49]:
df.printSchema()

root
 |-- selecao: string (nullable = true)
 |-- numero: integer (nullable = true)
 |-- posicao: string (nullable = true)
 |-- nome_fifa: string (nullable = true)
 |-- data_nasc: string (nullable = true)
 |-- nome_camisa: string (nullable = true)
 |-- time: string (nullable = true)
 |-- altura: integer (nullable = true)
 |-- peso: integer (nullable = true)
 |-- ano_nasc: integer (nullable = true)
 |-- dia_nasc: string (nullable = true)
 |-- mes_nasc: string (nullable = true)
 |-- nascimento: date (nullable = true)



In [48]:
df = df.withColumn('nascimento', col('nascimento').cast(DateType()))
df.show()

+---------+------+-------+------------------+----------+-----------+--------------------+------+----+--------+--------+--------+----------+
|  selecao|numero|posicao|         nome_fifa| data_nasc|nome_camisa|                time|altura|peso|ano_nasc|dia_nasc|mes_nasc|nascimento|
+---------+------+-------+------------------+----------+-----------+--------------------+------+----+--------+--------+--------+----------+
|Argentina|     3|     DF|TAGLIAFICO Nicolas|31.08.1992| TAGLIAFICO|      AFC Ajax (NED)|   169|  65|    1992|      31|      08|1992-08-31|
|Argentina|    22|     MF|    PAVON Cristian|21.01.1996|      PAVÓN|CA Boca Juniors (...|   169|  65|    1996|      21|      01|1996-01-21|
|Argentina|    15|     MF|    LANZINI Manuel|15.02.1993|    LANZINI|West Ham United F...|   167|  66|    1993|      15|      02|1993-02-15|
|Argentina|    18|     DF|    SALVIO Eduardo|13.07.1990|     SALVIO|    SL Benfica (POR)|   167|  69|    1990|      13|      07|1990-07-13|
|Argentina|    10|  

In [50]:
df.printSchema()

root
 |-- selecao: string (nullable = true)
 |-- numero: integer (nullable = true)
 |-- posicao: string (nullable = true)
 |-- nome_fifa: string (nullable = true)
 |-- data_nasc: string (nullable = true)
 |-- nome_camisa: string (nullable = true)
 |-- time: string (nullable = true)
 |-- altura: integer (nullable = true)
 |-- peso: integer (nullable = true)
 |-- ano_nasc: integer (nullable = true)
 |-- dia_nasc: string (nullable = true)
 |-- mes_nasc: string (nullable = true)
 |-- nascimento: date (nullable = true)



# Aula 2

# Importando função window

In [51]:
from pyspark.sql.window import Window

In [52]:
df.show()

+---------+------+-------+------------------+----------+-----------+--------------------+------+----+--------+--------+--------+----------+
|  selecao|numero|posicao|         nome_fifa| data_nasc|nome_camisa|                time|altura|peso|ano_nasc|dia_nasc|mes_nasc|nascimento|
+---------+------+-------+------------------+----------+-----------+--------------------+------+----+--------+--------+--------+----------+
|Argentina|     3|     DF|TAGLIAFICO Nicolas|31.08.1992| TAGLIAFICO|      AFC Ajax (NED)|   169|  65|    1992|      31|      08|1992-08-31|
|Argentina|    22|     MF|    PAVON Cristian|21.01.1996|      PAVÓN|CA Boca Juniors (...|   169|  65|    1996|      21|      01|1996-01-21|
|Argentina|    15|     MF|    LANZINI Manuel|15.02.1993|    LANZINI|West Ham United F...|   167|  66|    1993|      15|      02|1993-02-15|
|Argentina|    18|     DF|    SALVIO Eduardo|13.07.1990|     SALVIO|    SL Benfica (POR)|   167|  69|    1990|      13|      07|1990-07-13|
|Argentina|    10|  

# Drop de colunas

In [57]:
df = df.drop('data_nasc')
df.show()

+---------+------+-------+------------------+-----------+--------------------+------+----+--------+--------+--------+----------+
|  selecao|numero|posicao|         nome_fifa|nome_camisa|                time|altura|peso|ano_nasc|dia_nasc|mes_nasc|nascimento|
+---------+------+-------+------------------+-----------+--------------------+------+----+--------+--------+--------+----------+
|Argentina|     3|     DF|TAGLIAFICO Nicolas| TAGLIAFICO|      AFC Ajax (NED)|   169|  65|    1992|      31|      08|1992-08-31|
|Argentina|    22|     MF|    PAVON Cristian|      PAVÓN|CA Boca Juniors (...|   169|  65|    1996|      21|      01|1996-01-21|
|Argentina|    15|     MF|    LANZINI Manuel|    LANZINI|West Ham United F...|   167|  66|    1993|      15|      02|1993-02-15|
|Argentina|    18|     DF|    SALVIO Eduardo|     SALVIO|    SL Benfica (POR)|   167|  69|    1990|      13|      07|1990-07-13|
|Argentina|    10|     FW|      MESSI Lionel|      MESSI|  FC Barcelona (ESP)|   170|  72|    198

# Dataframe Backup

In [58]:
# é simples, só atribuir o primeiro dataframe para outra variável
df2 = df

# Window Ranking Functions<br>

* Window function 1: número de linhas - row_numbers()
* Window function 2: ranking 1 - rank()
* Window function 3: ranking 2 - dense_rank()
* Window function 4: porcentagem ranking - percent_rank()
* Window function 5: divisão em 'N' - ntile()

# Window function 1: número de linhas - row_numbers()

In [59]:
numero_linhas = Window.partitionBy('selecao').orderBy(desc('altura'))

df.withColumn('numero_linha', row_number().over(numero_linhas)).show(50)

+---------+------+-------+------------------+-----------+--------------------+------+----+--------+--------+--------+----------+------------+
|  selecao|numero|posicao|         nome_fifa|nome_camisa|                time|altura|peso|ano_nasc|dia_nasc|mes_nasc|nascimento|numero_linha|
+---------+------+-------+------------------+-----------+--------------------+------+----+--------+--------+--------+----------+------------+
|Argentina|     6|     DF|    FAZIO Federico|      FAZIO|       AS Roma (ITA)|   199|  85|    1987|      17|      03|1987-03-17|           1|
|Argentina|     1|     GK|     GUZMAN Nahuel|     GUZMÁN|   Tigres UANL (MEX)|   192|  90|    1986|      10|      02|1986-02-10|           2|
|Argentina|    16|     DF|       ROJO Marcos|       ROJO|Manchester United...|   189|  82|    1990|      20|      03|1990-03-20|           3|
|Argentina|    12|     GK|     ARMANI Franco|     ARMANI|CA River Plate (ARG)|   189|  85|    1986|      16|      10|1986-10-16|           4|
|Argen

# Window function 2: ranking 1 - rank()

In [60]:
rank1 = Window.partitionBy('selecao').orderBy(desc('altura'))

df.withColumn('rank', rank().over(rank1)).show(50)

+---------+------+-------+------------------+-----------+--------------------+------+----+--------+--------+--------+----------+----+
|  selecao|numero|posicao|         nome_fifa|nome_camisa|                time|altura|peso|ano_nasc|dia_nasc|mes_nasc|nascimento|rank|
+---------+------+-------+------------------+-----------+--------------------+------+----+--------+--------+--------+----------+----+
|Argentina|     6|     DF|    FAZIO Federico|      FAZIO|       AS Roma (ITA)|   199|  85|    1987|      17|      03|1987-03-17|   1|
|Argentina|     1|     GK|     GUZMAN Nahuel|     GUZMÁN|   Tigres UANL (MEX)|   192|  90|    1986|      10|      02|1986-02-10|   2|
|Argentina|    16|     DF|       ROJO Marcos|       ROJO|Manchester United...|   189|  82|    1990|      20|      03|1990-03-20|   3|
|Argentina|    12|     GK|     ARMANI Franco|     ARMANI|CA River Plate (ARG)|   189|  85|    1986|      16|      10|1986-10-16|   3|
|Argentina|    23|     GK|CABALLERO Wilfredo|  CABALLERO|    C

# Window function 3: ranking 2 - dense_rank()

In [61]:
rank2 = Window.partitionBy('selecao').orderBy(desc('altura'))
df.withColumn('rank2', dense_rank().over(rank2)).show(50)

+---------+------+-------+------------------+-----------+--------------------+------+----+--------+--------+--------+----------+-----+
|  selecao|numero|posicao|         nome_fifa|nome_camisa|                time|altura|peso|ano_nasc|dia_nasc|mes_nasc|nascimento|rank2|
+---------+------+-------+------------------+-----------+--------------------+------+----+--------+--------+--------+----------+-----+
|Argentina|     6|     DF|    FAZIO Federico|      FAZIO|       AS Roma (ITA)|   199|  85|    1987|      17|      03|1987-03-17|    1|
|Argentina|     1|     GK|     GUZMAN Nahuel|     GUZMÁN|   Tigres UANL (MEX)|   192|  90|    1986|      10|      02|1986-02-10|    2|
|Argentina|    16|     DF|       ROJO Marcos|       ROJO|Manchester United...|   189|  82|    1990|      20|      03|1990-03-20|    3|
|Argentina|    12|     GK|     ARMANI Franco|     ARMANI|CA River Plate (ARG)|   189|  85|    1986|      16|      10|1986-10-16|    3|
|Argentina|    23|     GK|CABALLERO Wilfredo|  CABALLER

# Window function 4: porcentagem ranking - percent_rank()

In [64]:
porcentagem = Window.partitionBy('selecao').orderBy(desc('altura'))
df.withColumn('%', percent_rank().over(porcentagem)).show(50)

+---------+------+-------+------------------+-----------+--------------------+------+----+--------+--------+--------+----------+--------------------+
|  selecao|numero|posicao|         nome_fifa|nome_camisa|                time|altura|peso|ano_nasc|dia_nasc|mes_nasc|nascimento|                   %|
+---------+------+-------+------------------+-----------+--------------------+------+----+--------+--------+--------+----------+--------------------+
|Argentina|     6|     DF|    FAZIO Federico|      FAZIO|       AS Roma (ITA)|   199|  85|    1987|      17|      03|1987-03-17|                 0.0|
|Argentina|     1|     GK|     GUZMAN Nahuel|     GUZMÁN|   Tigres UANL (MEX)|   192|  90|    1986|      10|      02|1986-02-10|0.045454545454545456|
|Argentina|    16|     DF|       ROJO Marcos|       ROJO|Manchester United...|   189|  82|    1990|      20|      03|1990-03-20| 0.09090909090909091|
|Argentina|    12|     GK|     ARMANI Franco|     ARMANI|CA River Plate (ARG)|   189|  85|    1986| 

# Window function 5: divisão em 'N' - ntile()

In [69]:
# efetua a divisão das linhas de dados de acordo com o partitionby

parte = Window.partitionBy('selecao').orderBy(desc('altura'))
df.withColumn('parte', ntile(5).over(parte)).show(50)

+---------+------+-------+------------------+-----------+--------------------+------+----+--------+--------+--------+----------+-----+
|  selecao|numero|posicao|         nome_fifa|nome_camisa|                time|altura|peso|ano_nasc|dia_nasc|mes_nasc|nascimento|parte|
+---------+------+-------+------------------+-----------+--------------------+------+----+--------+--------+--------+----------+-----+
|Argentina|     6|     DF|    FAZIO Federico|      FAZIO|       AS Roma (ITA)|   199|  85|    1987|      17|      03|1987-03-17|    1|
|Argentina|     1|     GK|     GUZMAN Nahuel|     GUZMÁN|   Tigres UANL (MEX)|   192|  90|    1986|      10|      02|1986-02-10|    1|
|Argentina|    16|     DF|       ROJO Marcos|       ROJO|Manchester United...|   189|  82|    1990|      20|      03|1990-03-20|    1|
|Argentina|    12|     GK|     ARMANI Franco|     ARMANI|CA River Plate (ARG)|   189|  85|    1986|      16|      10|1986-10-16|    1|
|Argentina|    23|     GK|CABALLERO Wilfredo|  CABALLER

# Window Analytic Functions

# Window function  LAG / Degrau - lag()

In [72]:
degrau = Window.partitionBy('selecao').orderBy(desc('altura'))
df.withColumn('degrau', lag(col('altura')).over(degrau)).show(50)

+---------+------+-------+------------------+-----------+--------------------+------+----+--------+--------+--------+----------+------+
|  selecao|numero|posicao|         nome_fifa|nome_camisa|                time|altura|peso|ano_nasc|dia_nasc|mes_nasc|nascimento|degrau|
+---------+------+-------+------------------+-----------+--------------------+------+----+--------+--------+--------+----------+------+
|Argentina|     6|     DF|    FAZIO Federico|      FAZIO|       AS Roma (ITA)|   199|  85|    1987|      17|      03|1987-03-17|  NULL|
|Argentina|     1|     GK|     GUZMAN Nahuel|     GUZMÁN|   Tigres UANL (MEX)|   192|  90|    1986|      10|      02|1986-02-10|   199|
|Argentina|    16|     DF|       ROJO Marcos|       ROJO|Manchester United...|   189|  82|    1990|      20|      03|1990-03-20|   192|
|Argentina|    12|     GK|     ARMANI Franco|     ARMANI|CA River Plate (ARG)|   189|  85|    1986|      16|      10|1986-10-16|   189|
|Argentina|    23|     GK|CABALLERO Wilfredo|  C

# Lead / Degrau - lead()

In [74]:
degrau2 = Window.partitionBy('selecao').orderBy(desc('altura'))
df.withColumn('degrau2', lead(col('altura')).over(degrau2)).show(50)

+---------+------+-------+------------------+-----------+--------------------+------+----+--------+--------+--------+----------+-------+
|  selecao|numero|posicao|         nome_fifa|nome_camisa|                time|altura|peso|ano_nasc|dia_nasc|mes_nasc|nascimento|degrau2|
+---------+------+-------+------------------+-----------+--------------------+------+----+--------+--------+--------+----------+-------+
|Argentina|     6|     DF|    FAZIO Federico|      FAZIO|       AS Roma (ITA)|   199|  85|    1987|      17|      03|1987-03-17|    192|
|Argentina|     1|     GK|     GUZMAN Nahuel|     GUZMÁN|   Tigres UANL (MEX)|   192|  90|    1986|      10|      02|1986-02-10|    189|
|Argentina|    16|     DF|       ROJO Marcos|       ROJO|Manchester United...|   189|  82|    1990|      20|      03|1990-03-20|    189|
|Argentina|    12|     GK|     ARMANI Franco|     ARMANI|CA River Plate (ARG)|   189|  85|    1986|      16|      10|1986-10-16|    186|
|Argentina|    23|     GK|CABALLERO Wilfr

# Agregações

# GroupBy + AGG 1

In [79]:
df.groupBy('selecao').agg({'altura': 'avg'}).orderBy('avg(altura)', ascending = False).show(50)

+--------------+------------------+
|       selecao|       avg(altura)|
+--------------+------------------+
|        Serbia|186.69565217391303|
|       Denmark| 186.6086956521739|
|       Germany| 185.7826086956522|
|        Sweden| 185.7391304347826|
|       Iceland|185.52173913043478|
|       Belgium|185.34782608695653|
|       Croatia| 185.2608695652174|
|       Nigeria|184.52173913043478|
|       IR Iran|184.47826086956522|
|        Russia| 184.3913043478261|
|       Senegal|183.65217391304347|
|        France|183.30434782608697|
|        Poland|183.17391304347825|
|       Tunisia|183.08695652173913|
|   Switzerland|182.91304347826087|
|       England| 182.7391304347826|
|       Morocco|182.69565217391303|
|        Panama|182.17391304347825|
|Korea Republic| 181.8695652173913|
|       Uruguay|181.04347826086956|
|         Egypt|             181.0|
|     Australia| 180.8695652173913|
|        Brazil| 180.7826086956522|
|      Colombia| 180.7826086956522|
|    Costa Rica|180.69565217

# GroupBy + AGG 2

In [80]:
df.groupBy('selecao').agg(max('altura')).orderBy('max(altura)', ascending = False).show(50)

+--------------+-----------+
|       selecao|max(altura)|
+--------------+-----------+
|       Croatia|        201|
|       Denmark|        200|
|     Argentina|        199|
|       Belgium|        199|
|        Sweden|        198|
|       Iceland|        198|
|        France|        197|
|       Nigeria|        197|
|Korea Republic|        197|
|        Panama|        197|
|        Russia|        196|
|       Senegal|        196|
|       Uruguay|        196|
|       England|        196|
|    Costa Rica|        196|
|       Germany|        195|
|        Brazil|        195|
|        Poland|        195|
|        Serbia|        195|
|       IR Iran|        194|
|         Spain|        194|
|         Egypt|        194|
|      Colombia|        194|
|     Australia|        193|
|       Tunisia|        192|
|  Saudi Arabia|        192|
|   Switzerland|        192|
|      Portugal|        191|
|       Morocco|        190|
|        Mexico|        190|
|          Peru|        189|
|         Japa

# Where

In [84]:
# where é igual ao filter
df.where('selecao = "Brazil"').where('posicao = "DF"').show()

# where((condicao 1) & where(condicao 2))

+-------+------+-------+-------------+-----------+--------------------+------+----+--------+--------+--------+----------+
|selecao|numero|posicao|    nome_fifa|nome_camisa|                time|altura|peso|ano_nasc|dia_nasc|mes_nasc|nascimento|
+-------+------+-------+-------------+-----------+--------------------+------+----+--------+--------+--------+----------+
| Brazil|    22|     DF|       FAGNER|     FAGNER|SC Corinthians (BRA)|   168|  67|    1989|      11|      06|1989-06-11|
| Brazil|     6|     DF|  FILIPE LUIS|FILIPE LUIS|Atletico Madrid (...|   182|  73|    1985|      09|      08|1985-08-09|
| Brazil|    13|     DF|   MARQUINHOS| MARQUINHOS|Paris Saint-Germa...|   183|  75|    1994|      14|      05|1994-05-14|
| Brazil|     3|     DF|      MIRANDA|    MIRANDA|FC Internazionale...|   186|  78|    1984|      07|      09|1984-09-07|
| Brazil|    14|     DF|       DANILO|     DANILO|Manchester City F...|   184|  78|    1991|      15|      07|1991-07-15|
| Brazil|     2|     DF|

In [97]:
top_1 = Window.partitionBy('selecao').orderBy(desc('altura'))
df.withColumn('top', row_number().over(top_1)).filter('top = "1"').show(50)

+--------------+------+-------+-------------------+-------------+--------------------+------+----+--------+--------+--------+----------+---+
|       selecao|numero|posicao|          nome_fifa|  nome_camisa|                time|altura|peso|ano_nasc|dia_nasc|mes_nasc|nascimento|top|
+--------------+------+-------+-------------------+-------------+--------------------+------+----+--------+--------+--------+----------+---+
|     Argentina|     6|     DF|     FAZIO Federico|        FAZIO|       AS Roma (ITA)|   199|  85|    1987|      17|      03|1987-03-17|  1|
|     Australia|    12|     GK|         JONES Brad|        JONES|Feyenoord Rotterd...|   193|  87|    1982|      19|      03|1982-03-19|  1|
|       Belgium|     1|     GK|   COURTOIS Thibaut|     COURTOIS|    Chelsea FC (ENG)|   199|  91|    1992|      11|      05|1992-05-11|  1|
|        Brazil|    16|     GK|             CASSIO|       CASSIO|SC Corinthians (BRA)|   195|  92|    1987|      06|      06|1987-06-06|  1|
|      Colomb

# Describe

In [98]:
df.describe().show()

+-------+---------+-----------------+-------+------------+-----------+--------------------+-----------------+-----------------+------------------+------------------+------------------+
|summary|  selecao|           numero|posicao|   nome_fifa|nome_camisa|                time|           altura|             peso|          ano_nasc|          dia_nasc|          mes_nasc|
+-------+---------+-----------------+-------+------------+-----------+--------------------+-----------------+-----------------+------------------+------------------+------------------+
|  count|      736|              736|    736|         736|        736|                 736|              736|              736|               736|               736|               736|
|   mean|     NULL|             12.0|   NULL|        NULL|       NULL|                NULL|182.4076086956522|77.18885869565217| 1990.110054347826|15.793478260869565|5.8790760869565215|
| stddev|     NULL|6.637760461599851|   NULL|        NULL|       NULL|     

In [105]:
df.where('selecao = "Brazil"').describe().show(50)

+-------+-------+-----------------+-------+---------+-----------+--------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|summary|selecao|           numero|posicao|nome_fifa|nome_camisa|                time|           altura|             peso|         ano_nasc|         dia_nasc|         mes_nasc|
+-------+-------+-----------------+-------+---------+-----------+--------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|  count|     23|               23|     23|       23|         23|                  23|               23|               23|               23|               23|               23|
|   mean|   NULL|             12.0|   NULL|     NULL|       NULL|                NULL|180.7826086956522|76.56521739130434|1989.391304347826|11.26086956521739|6.130434782608695|
| stddev|   NULL|6.782329983125267|   NULL|     NULL|       NULL|                NULL|7.354383490255254|8.239737898

# Função de agregação usando Window Function

In [109]:
parametro = Window.partitionBy('selecao').orderBy(desc('altura'))
parametro2 = Window.partitionBy('selecao')

df.withColumn('linhax', row_number().over(parametro))\
.withColumn('media', avg('altura').over(parametro2))\
.withColumn('maximo', max('altura').over(parametro2))\
.withColumn('minimo', min('altura').over(parametro2))\
.filter('linhax = "1"').select('selecao', 'media', 'maximo', 'minimo')\
.orderBy('media', ascending = False).show(50)

+--------------+------------------+------+------+
|       selecao|             media|maximo|minimo|
+--------------+------------------+------+------+
|        Serbia|186.69565217391303|   195|   169|
|       Denmark| 186.6086956521739|   200|   171|
|       Germany| 185.7826086956522|   195|   176|
|        Sweden| 185.7391304347826|   198|   177|
|       Iceland|185.52173913043478|   198|   170|
|       Belgium|185.34782608695653|   199|   169|
|       Croatia| 185.2608695652174|   201|   172|
|       Nigeria|184.52173913043478|   197|   172|
|       IR Iran|184.47826086956522|   194|   177|
|        Russia| 184.3913043478261|   196|   173|
|       Senegal|183.65217391304347|   196|   173|
|        France|183.30434782608697|   197|   168|
|        Poland|183.17391304347825|   195|   172|
|       Tunisia|183.08695652173913|   192|   170|
|   Switzerland|182.91304347826087|   192|   165|
|       England| 182.7391304347826|   196|   170|
|       Morocco|182.69565217391303|   190|   167|
